### 1-1. 데이터 준비(구글 스프레드시트)

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('nutest-a559bf1ec68b.json', scope)
client = gspread.authorize(creds)

In [3]:
sheet = client.open('sent2').sheet1
sent2 = sheet.get_all_records()
import pandas as pd
data = pd.DataFrame(sent2)
data.head()

,document,id,label
0,오늘 택배 보낼 건데 가능해요?,F100100001001,F100100001
1,오늘 택배기사 좀 바로 보내주세요,F100100001002,F100100001
2,택배 당일 수거 부탁드립니다,F100100001003,F100100001
3,반품할건데오늘바로되는지알려주세요,F100100001004,F100100001
4,당일 반품수거는 어떻게 하나요?,F100100001005,F100100001


_____________

### 1-2. 데이터 준비(네이버 지식인 택배 질문)

In [1]:
def delivery_reply(pageno):
    
    import urllib.request
    from bs4 import BeautifulSoup
    import numpy as np
    import pandas as pd

    pageno = pageno
    url = "http://kin.naver.com/search/list.nhn?query=%ED%83%9D%EB%B0%B0&page=" + str(pageno)
    f = urllib.request.urlopen(url)
    overlog = f.read() .decode("utf8")
    f.close()
    soup = BeautifulSoup(overlog, "lxml")
    
    date = list(lv.get_text().replace(".","") for lv in soup.find_all('dd', {'class': 'txt_inline'}))
    title = list(lv.get_text() for lv in soup.find_all('a', {'class': '_nclicks:kin.txt'}))
    category = list(lv.get_text() for lv in soup.find_all('a', {'class': 'txt_g1 _nclicks:kin.cat2'}))
    reply = list(lv.get_text().replace("답변수 ","") for lv in soup.find_all('span', {'class': 'hit'}))
    vote = list(lv.get_text().split('추천수 ')[1].split('\n')[0] for lv in soup.find_all('dd', {'class': 'txt_block'}))
    href = list(str(lv).split("href=")[1].split("tabindex")[0].split(">")[0].replace('"', "").replace(" ", "").replace("amp;","").replace("'","") for lv in soup.find_all('a', {'class': "_nclicks:kin.txt"})[0:])

    delivery = pd.DataFrame({'date':date, 'title':title, 'category':category, 'href':href, 'vote':vote, 'reply':reply})
    delivery = delivery[['date', 'title','category', 'reply', 'vote', 'href']]
    
    return (delivery)

In [6]:
def delivery_reply2(pageno2):
    
    delivery_dataset=[]
    
    for i in range(pageno2):
        delivery_dataset.append(delivery_reply(i))
    
    delivery_fulldata = pd.concat(delivery_dataset)
    delivery_fulldata.reset_index()
    delivery_fulldata.to_csv('delivery_fulldata.csv')
    
    return (fulldata)

In [4]:
import pandas as pd
#pd.set_option('max_colwidth',200)
pd.set_option('max_colwidth',60)

In [7]:
delivery_reply2(5)

,date,title,category,reply,vote,href
0,20170711,택배를 도둑맞았습니다...,절도,2,0,http://kin.naver.com/qna/detail.nhn?d1id=6&dirId=6020604...
1,20170712,택배를 분실하게 되면 책임은 누가??,"소비자관련법, 상법",1,0,http://kin.naver.com/qna/detail.nhn?d1id=6&dirId=60207&d...
2,20170716,우체국택배,택배,2,6,http://kin.naver.com/qna/detail.nhn?d1id=8&dirId=81302&d...
3,20170716,우체국택배 배송,택배,4,5,http://kin.naver.com/qna/detail.nhn?d1id=8&dirId=81302&d...
4,20170715,대한통운 택배조회,택배,3,5,http://kin.naver.com/qna/detail.nhn?d1id=8&dirId=81302&d...
5,20170714,롯데택배 관련,택배,1,0,http://kin.naver.com/qna/detail.nhn?d1id=8&dirId=81302&d...
6,20170711,택배 배송일,택배,1,0,http://kin.naver.com/qna/detail.nhn?d1id=8&dirId=81302&d...
7,20170715,CJ GLS 택배,택배,1,1,http://kin.naver.com/qna/detail.nhn?d1id=8&dirId=81302&d...
8,20170714,택배내일 올까요?,택배,2,0,http://kin.naver.com/qna/detail.nhn?d1id=8&dirId=81302&d...
9,20170715,한진택배 배송,택배,2,0,http://kin.naver.com/qna/detail.nhn?d1id=8&dirId=81302&d...


In [10]:
delivery_df = pd.read_csv('delivery_fulldata.csv')

0          택배를 도둑맞았습니다...
1    택배를 분실하게 되면 책임은 누가??
2                   우체국택배
3                우체국택배 배송
4               대한통운 택배조회
Name: title, dtype: object

### 2. 형태소 분석, Mecab

In [14]:
from konlpy.tag import Mecab
mecab = Mecab()

#### 품사태그
![title](mecab_Tag.jpg)
https://docs.google.com/spreadsheets/d/1-9blXKjtjeKZqsf4NzHeYJCrr49-nXeRF6D80udfcwY/edit#gid=589544265
#### 사용자 사전 추가 방법
https://bitbucket.org/eunjeon/mecab-ko-dic/src/5fad4609d23a1b172a57e23addfe167ac5f02bf1/final/user-dic/README.md?fileviewer=file-view-default

In [15]:
print(mecab.morphs(u'영등포구청역에 있는 맛집 좀 알려주세요.')),
print(mecab.nouns(u'우리나라에는 무릎 치료를 잘하는 정형외과가 없는가!')),
print(mecab.pos(u'자연주의 쇼핑몰은 어떤 곳인가?'))

['영등포구청역', '에', '있', '는', '맛집', '좀', '알려', '주', '세요', '.']
['우리', '나라', '무릎', '치료', '정형외과']
[('자연주의', 'NNG'), ('쇼핑몰', 'NNG'), ('은', 'JX'), ('어떤', 'MM'), ('곳', 'NNG'), ('인가', 'VCP+EF'), ('?', 'SF')]


In [23]:
delivery_mecab = [mecab.pos(row) for row in delivery_df['title']]

In [25]:
delivery_mecab[:5]

[[('택배', 'NNG'),
  ('를', 'JKO'),
  ('도둑맞', 'VV'),
  ('았', 'EP'),
  ('습니다', 'EF'),
  ('.', 'SF'),
  ('..', 'SY')],
 [('택배', 'NNG'),
  ('를', 'JKO'),
  ('분실', 'NNG'),
  ('하', 'XSV'),
  ('게', 'EC'),
  ('되', 'VV'),
  ('면', 'EC'),
  ('책임', 'NNG'),
  ('은', 'JX'),
  ('누가', 'NP+JKS'),
  ('?', 'SF'),
  ('?', 'SF')],
 [('우체국', 'NNG'), ('택배', 'NNG')],
 [('우체국', 'NNG'), ('택배', 'NNG'), ('배송', 'NNG')],
 [('대한', 'VV+ETM'), ('통운', 'NNG'), ('택배', 'NNG'), ('조회', 'NNG')]]